This notebook is basically a scratchpad for some quick analyses or testing.

In [1]:
%load_ext autoreload

In [8]:
%autoreload

import audl_advanced_stats as audl
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns

In [9]:
%%time
s = audl.Season(upload=False, download=False)
s.get_players()
s.get_teams()
s.get_games(small_file=False)
s.get_games(small_file=True)
# s.get_game_qc()
s.get_player_stats_by_game()
s.get_player_stats_by_season()

Wall time: 7.24 s


,playerid,name,team,year,games,total_points,o_points,o_point_scores,o_point_noturns,d_points,...,receptions_dump_pct,completion_huck_pct,attempts_huck_pct,receptions_huck_pct,completion_swing_pct,attempts_swing_pct,receptions_swing_pct,completion_throw_pct,attempts_throw_pct,receptions_throw_pct
0,8663,Andrew Meshnick,MAD,2021.0,9.0,186.0,44.0,23.0,25.0,142.0,...,0.064516,0.857143,0.111111,0.000000,1.000000,0.111111,0.161290,0.880000,0.396825,0.451613
1,8664,Henry Goldenberg,MAD,2021.0,12.0,120.0,56.0,34.0,27.0,64.0,...,0.076923,0.571429,0.054688,0.019231,0.966667,0.234375,0.201923,1.000000,0.234375,0.192308
2,8665,Sean Treacy,MAD,2021.0,2.0,31.0,2.0,2.0,1.0,29.0,...,0.000000,1.000000,0.083333,0.000000,1.000000,0.166667,0.142857,0.800000,0.416667,0.357143
3,8667,Tom Annen,MAD,2021.0,6.0,68.0,27.0,17.0,14.0,41.0,...,0.136986,NaN,0.000000,0.000000,1.000000,0.177083,0.246575,0.945946,0.385417,0.068493
4,8668,Joshua Wilson,MAD,2021.0,12.0,224.0,45.0,29.0,23.0,179.0,...,0.126316,0.625000,0.032129,0.000000,1.000000,0.152610,0.242105,0.961165,0.413655,0.194737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737,9717,Daniel Mah,SD,2021.0,1.0,12.0,0.0,0.0,0.0,12.0,...,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.333333
738,9719,Elliot Warner,SD,2021.0,1.0,13.0,0.0,0.0,0.0,13.0,...,0.000000,NaN,0.000000,0.000000,NaN,0.000000,0.500000,NaN,0.000000,0.500000
739,9720,Grady Watkins,SD,2021.0,1.0,12.0,0.0,0.0,0.0,12.0,...,0.000000,NaN,0.000000,0.000000,1.000000,0.250000,0.000000,1.000000,0.500000,0.666667
740,9722,Henry Ing,PIT,2021.0,1.0,20.0,3.0,0.0,2.0,17.0,...,0.000000,1.000000,0.166667,0.000000,1.000000,0.333333,0.166667,1.000000,0.166667,0.166667


# Scores vs Time Remaining

In [102]:
def sec_to_min(seconds):
    minutes = int(seconds/60)
    seconds_left = seconds%60
    return f"{minutes}:{seconds_left:02d}"
    
def bin_labels(bins):
    return [f"{sec_to_min(bins[i+1])}-{sec_to_min(x+1)}" for i, x in enumerate(bins[:-1])]

s = audl.Season(upload=False, download=False)
bins = [-1, 10, 20, 30, 40, 50, 60, 120, 240, 480, 720]
labels = bin_labels(bins)
df = (
    s.get_games(small_file=False)
    .query("t==1")
#     .query("period==[1,2,3,]")
    .assign(s_before_bucket=lambda x: pd.Categorical(pd.cut(x["s_before"],
                                                            bins=bins,
                                                            labels=labels),
                                                     categories=reversed(labels),
                                                     ordered=True))
    .groupby(["s_before_bucket", "possession_outcome_general"])
    .size()
    .groupby(level=["s_before_bucket"])
    .apply(lambda x: x/x.sum())
    .rename("pct")
    .reset_index()
    .query("possession_outcome_general=='Score'")
#     .plot(x="s_before_bucket", y="pct")
#     [["game_id", "team_id", "t", "event_name", "event_number",
#       "period", "s", "s_before", "s_after", "s_before_bucket", "possession_outcome_general"]]
    .rename(columns={
        "s_before_bucket": "Time at Start of Possession",
        "pct": "Score %"
    })
)
px.line(df, x="Time at Start of Possession", y="Score %")

# Pull Analysis

In [99]:
s = audl.Season(upload=False, download=False)
df = (
    s.get_games(small_file=False)
    .query("t==[3,4,]")
    .query("s_before>60")
    .assign(sideline=lambda x: np.where(x["x_after"].abs() > 16, "Outer Fifths", "Inner 3/5"),
            short=lambda x: np.where(x["y_after"]<80, "Shorter Than Brick", "As Deep As Brick"),
            pull_type=lambda x: x["event_name"]+", "+x["sideline"]+", "+x["short"],
           )
#     .query("period==[1,2,3,]")
    .groupby(["pull_type", "possession_outcome_general"])
    .size()
    .groupby(level=["pull_type",])
    .apply(lambda x: x/x.sum())
    .rename("pct")
    .reset_index()
    .query("possession_outcome_general=='Score'")
    .rename(columns={
        "pull_type": "Pull",
        "pct": "Score %"
    })
)
df
px.bar(df, x="Pull", y="Score %")

In [85]:
def pull_bin_labels(bins):
    return [f"{x}-{bins[i+1]}" for i, x in enumerate(bins[:-1])]

s = audl.Season(upload=False, download=False)
bins = list(range(11))
labels = pull_bin_labels(bins)
df = (
    s.get_games(small_file=False)
    .query("t==[3,4,]")
    .query("s_before>60")
#     .query("period==[1,2,3,]")
    .assign(hangtime_bucket=lambda x: pd.Categorical(pd.cut(x["hangtime"],
                                                            bins=bins,
                                                            labels=labels),
                                                     categories=labels,
                                                     ordered=True))
    .groupby(["event_name", "hangtime_bucket", "possession_outcome_general"])
    .size()
    .groupby(level=["event_name", "hangtime_bucket"])
    .apply(lambda x: x/x.sum())
    .rename("pct")
    .reset_index()
    .query("possession_outcome_general=='Score'")
    .query("event_name=='In-bounds Pull'")
    .rename(columns={
        "hangtime_bucket": "Pull Hangtime",
        "pct": "Score %"
    })
)
df
px.bar(df, x="Pull Hangtime", y="Score %")